In [1]:
import numpy as np
import tensorflow as tf
import keras
import keras_hub
import tensorflow_datasets as tfds

2025-11-05 14:36:21.055880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762349781.067470   11581 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762349781.070978   11581 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762349781.081028   11581 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762349781.081036   11581 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762349781.081038   11581 computation_placer.cc:177] computation placer alr

In [2]:
snli_train = tfds.load("snli", split="train[:20%]")
snli_val = tfds.load("snli", split="validation")
snli_test = tfds.load("snli", split="test")

# Here's an example of how our training samples look like, where we randomly select
# four samples:
sample = snli_test.batch(4).take(1).get_single_element()
sample

I0000 00:00:1762349783.791753   11581 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9128 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:08:00.0, compute capability: 7.5
2025-11-05 14:36:23.995598: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-11-05 14:36:23.996658: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


{'hypothesis': <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl is entertaining on stage',
        b'A group of people posing in front of a body of water.',
        b"The group of people aren't inide of the building.",
        b'The people are taking a carriage ride.'], dtype=object)>,
 'label': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 0])>,
 'premise': <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl in a blue leotard hula hoops on a stage with balloon shapes in the background.',
        b'A group of people taking pictures on a walkway in front of a large body of water.',
        b'Many people standing outside of a place talking to each other in front of a building that has a sign that says "HI-POINTE."',
        b'Three people are riding a carriage pulled by four horses.'],
       dtype=object)>}

In [3]:
def filter_labels(sample):
    return sample["label"] >= 0

In [4]:
def split_labels(sample):
    x = (sample["hypothesis"], sample["premise"])
    y = sample["label"]
    return x, y


train_ds = (
    snli_train.filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)
val_ds = (
    snli_val.filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)
test_ds = (
    snli_test.filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)

In [5]:
bert_classifier = keras_hub.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3
)

In [6]:
bert_classifier.fit(train_ds, validation_data=val_ds, epochs=1)

I0000 00:00:1762349789.411035   11693 service.cc:152] XLA service 0x7baa100020b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762349789.411052   11693 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2025-11-05 14:36:29.499922: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1762349790.000985   11693 cuda_dnn.cc:529] Loaded cuDNN version 91400


     16/Unknown 8s 11ms/step - loss: 1.1408 - sparse_categorical_accuracy: 0.2986

I0000 00:00:1762349793.501542   11693 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   6867/Unknown 95s 13ms/step - loss: 0.8636 - sparse_categorical_accuracy: 0.5994

2025-11-05 14:38:01.094375: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-11-05 14:38:01.094393: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2025-11-05 14:38:01.094401: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 9608663296477858221
2025-11-05 14:38:01.094405: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5886875488915500295
/home/xy/Desktop/ml/legacy/legacy/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` funct

6867/6867 ━━━━━━━━━━━━━━━━━━━━ 99s 13ms/step - loss: 0.7759 - sparse_categorical_accuracy: 0.6602 - val_loss: 0.5865 - val_sparse_categorical_accuracy: 0.7611


2025-11-05 14:38:05.162667: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2025-11-05 14:38:05.162681: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 9608663296477858221
2025-11-05 14:38:05.162685: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5886875488915500295
2025-11-05 14:38:05.162701: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 14121035944600323849


In [7]:
bert_classifier.evaluate(test_ds)

614/614 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.5918 - sparse_categorical_accuracy: 0.7594


2025-11-05 14:38:08.308326: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 9608663296477858221
2025-11-05 14:38:08.308341: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5886875488915500295
2025-11-05 14:38:08.308349: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 14121035944600323849


[0.5917884707450867, 0.7593648433685303]

In [8]:
bert_classifier = keras_hub.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3
)
bert_classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(5e-5),
    metrics=["accuracy"],
)

bert_classifier.fit(train_ds, validation_data=val_ds, epochs=1)
bert_classifier.evaluate(test_ds)

   6867/Unknown 102s 14ms/step - accuracy: 0.6027 - loss: 0.8622

2025-11-05 14:39:51.449811: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


6867/6867 ━━━━━━━━━━━━━━━━━━━━ 106s 14ms/step - accuracy: 0.6614 - loss: 0.7757 - val_accuracy: 0.7647 - val_loss: 0.5807
614/614 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7668 - loss: 0.5823


[0.582295298576355, 0.7667955756187439]

In [9]:
class TriangularSchedule(keras.optimizers.schedules.LearningRateSchedule):
    """Linear ramp up for `warmup` steps, then linear decay to zero at `total` steps."""

    def __init__(self, rate, warmup, total):
        self.rate = rate
        self.warmup = warmup
        self.total = total

    def get_config(self):
        config = {"rate": self.rate, "warmup": self.warmup, "total": self.total}
        return config

    def __call__(self, step):
        step = keras.ops.cast(step, dtype="float32")
        rate = keras.ops.cast(self.rate, dtype="float32")
        warmup = keras.ops.cast(self.warmup, dtype="float32")
        total = keras.ops.cast(self.total, dtype="float32")

        warmup_rate = rate * step / self.warmup
        cooldown_rate = rate * (total - step) / (total - warmup)
        triangular_rate = keras.ops.minimum(warmup_rate, cooldown_rate)
        return keras.ops.maximum(triangular_rate, 0.0)


bert_classifier = keras_hub.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3
)

# Get the total count of training batches.
# This requires walking the dataset to filter all -1 labels.
epochs = 3
total_steps = sum(1 for _ in train_ds.as_numpy_iterator()) * epochs
warmup_steps = int(total_steps * 0.2)

bert_classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(
        TriangularSchedule(1e-4, warmup_steps, total_steps)
    ),
    metrics=["accuracy"],
)

bert_classifier.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
   6867/Unknown 101s 14ms/step - accuracy: 0.5336 - loss: 0.9328

2025-11-05 14:41:42.168402: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 14121035944600323849


6867/6867 ━━━━━━━━━━━━━━━━━━━━ 105s 14ms/step - accuracy: 0.6306 - loss: 0.8120 - val_accuracy: 0.7599 - val_loss: 0.5830
Epoch 2/3


2025-11-05 14:41:46.137530: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6867/6867 ━━━━━━━━━━━━━━━━━━━━ 96s 14ms/step - accuracy: 0.7437 - loss: 0.6243 - val_accuracy: 0.7850 - val_loss: 0.5343
Epoch 3/3
6867/6867 ━━━━━━━━━━━━━━━━━━━━ 98s 14ms/step - accuracy: 0.7778 - loss: 0.5554 - val_accuracy: 0.7922 - val_loss: 0.5203


In [10]:
bert_classifier.evaluate(test_ds)

614/614 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7908 - loss: 0.5236


2025-11-05 14:45:03.456125: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 9608663296477858221
2025-11-05 14:45:03.456139: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 7713872062278699819
2025-11-05 14:45:03.456147: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5886875488915500295


[0.5235857367515564, 0.7908183932304382]

In [11]:
bert_classifier.save("bert_classifier.keras")
restored_model = keras.models.load_model("bert_classifier.keras")
restored_model.evaluate(test_ds)

/home/xy/Desktop/ml/legacy/legacy/lib/python3.10/site-packages/keras/src/saving/serialization_lib.py:749: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/home/xy/Desktop/ml/legacy/legacy/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 83 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


614/614 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.5236 - sparse_categorical_accuracy: 0.7908


[0.5235857367515564, 0.7908183932304382]

In [12]:
# Convert to Hypothesis-Premise pair, for forward pass through model
sample = (sample["hypothesis"], sample["premise"])
sample

(<tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl is entertaining on stage',
        b'A group of people posing in front of a body of water.',
        b"The group of people aren't inide of the building.",
        b'The people are taking a carriage ride.'], dtype=object)>,
 <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A girl in a blue leotard hula hoops on a stage with balloon shapes in the background.',
        b'A group of people taking pictures on a walkway in front of a large body of water.',
        b'Many people standing outside of a place talking to each other in front of a building that has a sign that says "HI-POINTE."',
        b'Three people are riding a carriage pulled by four horses.'],
       dtype=object)>)

In [13]:
predictions = bert_classifier.predict(sample)


def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=0)


# Get the class predictions with maximum probabilities
predictions = softmax(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   


In [14]:
# Inittializing a RoBERTa from preset
roberta_classifier = keras_hub.models.RobertaClassifier.from_preset(
    "roberta_base_en", num_classes=3, batch_size=1
)

roberta_classifier.fit(train_ds, validation_data=val_ds, epochs=1)

roberta_classifier.evaluate(test_ds)

2025-11-05 14:45:43.464503: W external/local_xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3021] Can't reduce memory use below 6.70GiB (7195519093 bytes) by rematerialization; only reduced to 9.84GiB (10561415240 bytes), down from 12.51GiB (13437631176 bytes) originally
2025-11-05 14:45:58.505683: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.50GiB (rounded to 9128689408)requested by op 
2025-11-05 14:45:58.505720: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-11-05 14:45:58.505726: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 119, Chunks in use: 119. 29.8KiB allocated for chunks. 29.8KiB in use in bin. 1.5KiB client-requested in use in bin.
2025-11-05 14:45:58.505730: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 147, Chunks in use: 147. 75.0KiB allocated for c

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/xy/Desktop/ml/legacy/legacy/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/xy/Desktop/ml/legacy/legacy/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/xy/Desktop/ml/legacy/legacy/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 758, in start

  File "/home/xy/Desktop/ml/legacy/legacy/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/xy/Desktop/ml

In [ ]:
predictions = roberta_classifier.predict(sample)
print(tf.math.argmax(predictions, axis=1).numpy())